In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

### Entropy Balancing

#### Outcome: Physical activity

In [62]:
dfg= pd.read_csv("testdata1/2.PA_treatment.csv")
df = pd.read_csv("testdata1/2.PA_control.csv")

In [63]:
df.columns

Index(['Age', 'Gender', 'BMI', 'Depression', 'Employment_status',
       'Baseline_Physical_Activity', 'Baseline_Pain',
       'Baseline_Quality_of_life', 'PA_change'],
      dtype='object')

In [64]:
dfg.columns

Index(['Age', 'Gender', 'BMI', 'Depression', 'Employment_status',
       'Baseline_Physical_Activity', 'Baseline_Pain',
       'Baseline_Quality_of_life', 'PA_change'],
      dtype='object')

In [65]:
df.shape

(1156, 9)

In [66]:
dfg.shape

(7603, 9)

In [67]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Pain,Baseline_Quality_of_life,PA_change
0,68.0,1.0,31.6,1.0,1.0,1,30.0,56.25,0
1,45.0,2.0,32.3,4.0,2.0,1,90.0,0.00,0
2,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1
3,46.0,1.0,30.0,1.0,1.0,1,20.0,68.75,0
4,75.0,1.0,27.8,1.0,3.0,1,90.0,56.25,0
...,...,...,...,...,...,...,...,...,...
1151,57.0,2.0,31.8,1.0,1.0,3,80.0,50.00,0
1152,67.0,1.0,23.1,2.0,1.0,2,70.0,75.00,1
1153,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0
1154,59.0,1.0,34.1,1.0,1.0,3,90.0,37.50,0


In [68]:
df.PA_change.value_counts()

PA_change
 0    708
-1    188
 1    147
-2     57
 2     56
Name: count, dtype: int64

In [69]:
# Add treatment indicator
df['Treatment'] = 0  # Control group
dfg['Treatment'] = 1  # Treatment group

# Combine the datasets
combined_data = pd.concat([df, dfg], ignore_index=True)

# Define covariates and outcome
# Include 'Baseline_Physical_Activity' for calculating weights
covariates_for_weighting = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
# Exclude 'Baseline_Physical_Activity' and 'PA_change' in the final balancing
covariates_for_balancing = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 
                            'Baseline_Physical_Activity','Baseline_Quality_of_life', 'Baseline_Pain']
outcome = 'PA_change'
treatment = 'Treatment'

# Separate treatment and control groups
control_data = combined_data[combined_data[treatment] == 0]
treatment_data = combined_data[combined_data[treatment] == 1]

# Calculate target moments (means) from the treatment group
target_means = treatment_data[covariates_for_weighting].mean()

# Define the objective function for optimization
def objective(weights):
    # Apply weights to the control group's covariates
    weighted_covariates = control_data[covariates_for_weighting].multiply(weights, axis=0)
    # Calculate the weighted means of the covariates
    weighted_means = weighted_covariates.mean()
    # Objective is to minimize the sum of squared differences between weighted means and target means
    return np.sum((weighted_means - target_means) ** 2)

# Initial weights (starting with equal weights for all control group observations)
initial_weights = np.ones(len(control_data))

# Constraints: weights should sum to the number of control units
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - len(control_data)})

# Bounds: weights should be non-negative
bounds = [(0, None) for _ in range(len(control_data))]

# Optimize weights using Sequential Least Squares Programming (SLSQP)
result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights from the result
optimized_weights = result.x

# Ensure weights are positive and normalized to sum to the number of control units
control_data['weights'] = np.abs(optimized_weights)
control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)

# Apply the weights to the control data covariates for balancing check, excluding Baseline_Physical_Activity and PA_change
weighted_control_data = control_data[covariates_for_balancing].multiply(control_data['weights'], axis=0)
weighted_control_data['Baseline_Physical_Activity'] = control_data['Baseline_Physical_Activity']
weighted_control_data['PA_change'] = control_data['PA_change']

# Calculate the weighted means of the covariates in the control group
weighted_means = weighted_control_data[covariates_for_balancing].mean()
print("Weighted Control Group Means:\n", weighted_means)
print("\nTarget Treatment Group Means:\n", target_means[covariates_for_balancing])

# For the final balanced control group, we use the repetitions approach excluding Baseline_Physical_Activity and PA_change
control_data['reps'] = np.round(control_data['weights']).astype(int)

# Adjust repetitions to match the original control group size
while control_data['reps'].sum() != len(control_data):
    diff = len(control_data) - control_data['reps'].sum()
    idx = np.random.choice(control_data.index)
    control_data.loc[idx, 'reps'] += np.sign(diff)

# Create the balanced control group by repeating rows according to the reps
repeated_rows = control_data.index.repeat(control_data['reps'])
balanced_control_data = control_data.loc[repeated_rows].reset_index(drop=True)

# Ensure Baseline_Physical_Activity remains unchanged
balanced_control_data['Baseline_Physical_Activity'] = np.repeat(control_data['Baseline_Physical_Activity'].values, control_data['reps'].values)

# Remove the temporary columns used for weighting
balanced_control_data = balanced_control_data.drop(columns=['weights', 'reps'])

# Check the means of the covariates to ensure they match the target means
balanced_means = balanced_control_data[covariates_for_balancing].mean()
print("Balanced Control Group Means:\n", balanced_means)
print("\nTarget Treatment Group Means (for balancing):\n", target_means[covariates_for_balancing])

# Combine the balanced control group with the treatment group
balanced_combined_data = pd.concat([balanced_control_data, treatment_data], ignore_index=True)

# Display the balanced control group dataframe
print("Balanced Control Group Data:")
print(balanced_control_data)

print("balanced_control_data.shape", balanced_control_data.shape)
print("control_data.shape", df.shape)

Weighted Control Group Means:
 Age                           56.693414
Gender                         1.648353
BMI                           29.594480
Depression                     1.120660
Employment_status              1.167799
Baseline_Physical_Activity     1.766436
Baseline_Quality_of_life      45.056591
Baseline_Pain                 47.219555
dtype: float64

Target Treatment Group Means:
 Age                           56.692227
Gender                         1.700250
BMI                           29.592791
Depression                     1.069578
Employment_status              1.156649
Baseline_Physical_Activity     2.047481
Baseline_Quality_of_life      45.055406
Baseline_Pain                 47.219387
dtype: float64
Balanced Control Group Means:
 Age                           56.562284
Gender                         1.643599
BMI                           29.577422
Depression                     1.117647
Employment_status              1.142734
Baseline_Physical_Activity     2.064

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_28375/956911863.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = np.abs(optimized_weights)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_28375/956911863.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['weights'] = control_data['weights'] / np.sum(control_data['weights']) * len(control_data)
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_28375/956911863.py:64: SettingWithCopyWar

In [70]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Pain,Baseline_Quality_of_life,PA_change,Treatment
0,68.0,1.0,31.6,1.0,1.0,1,30.0,56.25,0,0
1,45.0,2.0,32.3,4.0,2.0,1,90.0,0.00,0,0
2,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
3,46.0,1.0,30.0,1.0,1.0,1,20.0,68.75,0,0
4,75.0,1.0,27.8,1.0,3.0,1,90.0,56.25,0,0
...,...,...,...,...,...,...,...,...,...,...
1151,57.0,2.0,31.8,1.0,1.0,3,80.0,50.00,0,0
1152,67.0,1.0,23.1,2.0,1.0,2,70.0,75.00,1,0
1153,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1154,59.0,1.0,34.1,1.0,1.0,3,90.0,37.50,0,0


In [71]:
balanced_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Pain,Baseline_Quality_of_life,PA_change,Treatment
0,68.0,1.0,31.6,1.0,1.0,1,30.0,56.25,0,0
1,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
2,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
3,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
4,46.0,1.0,30.0,1.0,1.0,1,20.0,68.75,0,0
...,...,...,...,...,...,...,...,...,...,...
1151,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1152,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1153,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1154,59.0,1.0,34.1,1.0,1.0,3,90.0,37.50,0,0


In [72]:
weighted_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change
0,5.156463e+01,7.583034e-01,2.396239e+01,7.583034e-01,7.583034e-01,1,4.265456e+01,2.274910e+01,0
1,2.594004e-02,1.152890e-03,1.861918e-02,2.305781e-03,1.152890e-03,1,0.000000e+00,5.188007e-02,0
2,1.505910e+02,5.104778e+00,6.840403e+01,2.552389e+00,2.552389e+00,2,9.571459e+01,1.786672e+02,-1
3,6.575437e+01,1.429443e+00,4.288328e+01,1.429443e+00,1.429443e+00,1,9.827419e+01,2.858885e+01,0
4,2.642743e-16,3.523657e-18,9.795767e-17,3.523657e-18,1.057097e-17,1,1.982057e-16,3.171291e-16,0
...,...,...,...,...,...,...,...,...,...
1151,8.385454e+01,2.942264e+00,4.678201e+01,1.471132e+00,1.471132e+00,3,7.355661e+01,1.176906e+02,0
1152,1.157524e-01,1.727648e-03,3.990867e-02,3.455296e-03,1.727648e-03,2,1.295736e-01,1.209354e-01,1
1153,2.061968e+02,6.760550e+00,1.122251e+02,3.380275e+00,3.380275e+00,3,1.267603e+02,1.014082e+02,0
1154,3.425159e+01,5.805354e-01,1.979626e+01,5.805354e-01,5.805354e-01,3,2.177008e+01,5.224818e+01,0


In [73]:
# Control group (Real one)
mean_age_control = df['Age'].mean()
mean_gender_control = df['Gender'].mean()
mean_bmi_control = df['BMI'].mean()
mean_depression_control = df['Depression'].mean()
mean_employment_status_control = df['Employment_status'].mean()
mean_quality_of_life_control = df['Baseline_Quality_of_life'].mean()
mean_pain_control = df['Baseline_Pain'].mean()
mean_PA_control = df['Baseline_Physical_Activity'].mean()

# Weighted Control group 
weighted_mean_age_control = weighted_control_data['Age'].mean()
weighted_mean_gender_control = weighted_control_data['Gender'].mean()
weighted_mean_bmi_control = weighted_control_data['BMI'].mean()
weighted_mean_depression_control = weighted_control_data['Depression'].mean()
weighted_mean_employment_status_control = weighted_control_data['Employment_status'].mean()
weighted_mean_quality_of_life_control = weighted_control_data['Baseline_Quality_of_life'].mean()
weighted_mean_pain_control = weighted_control_data['Baseline_Pain'].mean()
weighted_mean_PA_control = weighted_control_data['Baseline_Physical_Activity'].mean()

# Balanced Control group
balanced_mean_age_control = balanced_control_data['Age'].mean()
balanced_mean_gender_control = balanced_control_data['Gender'].mean()
balanced_mean_bmi_control = balanced_control_data['BMI'].mean()
balanced_mean_depression_control = balanced_control_data['Depression'].mean()
balanced_mean_employment_status_control = balanced_control_data['Employment_status'].mean()
balanced_mean_quality_of_life_control = balanced_control_data['Baseline_Quality_of_life'].mean()
balanced_mean_pain_control = balanced_control_data['Baseline_Pain'].mean()
balanced_mean_PA_control = balanced_control_data['Baseline_Physical_Activity'].mean()

# Treatment group (unweighted)
mean_age_treatment = dfg['Age'].mean()
mean_gender_treatment = dfg['Gender'].mean()
mean_bmi_treatment = dfg['BMI'].mean()
mean_Depression_treatment = dfg['Depression'].mean()
mean_employment_status_treatment = dfg['Employment_status'].mean()
mean_QOL_treatment = dfg['Baseline_Quality_of_life'].mean()
mean_pain_treatment = dfg['Baseline_Pain'].mean()
mean_PA_treatment = dfg['Baseline_Physical_Activity'].mean()

# Result
print("Mean in control group:")
print(f"Mean Age (Control Group): {mean_age_control}")
print(f"Mean Gender (Control Group): {mean_gender_treatment}")
print(f"Mean BMI (Control Group): {mean_bmi_control}")
print(f"Mean Depression (Control Group): {mean_depression_control}")
print(f"Mean Employment Status (Control Group): {mean_employment_status_control}")
print(f"Mean Physical Activity (Control Group): {mean_PA_control}")
print(f"Mean QOL (Control Group): {mean_quality_of_life_control}")
print(f"Mean Pain (Control Group): {mean_pain_control}")



print("--------------------------------------------------")
print("Weighted mean in control group:")
print(f"Weighted Mean Age (Control Group): {weighted_mean_age_control}")
print(f"Weighted Mean Gender (Control Group): {weighted_mean_gender_control}")
print(f"Weighted Mean BMI (Control Group): {weighted_mean_bmi_control}")
print(f"Weighted Mean Depression (Control Group): {weighted_mean_depression_control}")
print(f"Weighted Mean Employment Status (Control Group): {weighted_mean_employment_status_control}")
print(f"Weighted Mean PA (Control Group): {weighted_mean_PA_control}")
print(f"Weighted Mean QOL (Control Group): {weighted_mean_quality_of_life_control}")
print(f"Weighted Mean Pain (Control Group): {weighted_mean_pain_control}")


print("--------------------------------------------------")
print("Balanced mean in control group:")
print(f"Balanced Mean Age (Control Group): {balanced_mean_age_control}")
print(f"Balanced Mean Gender (Control Group): {balanced_mean_gender_control}")
print(f"Balanced Mean BMI (Control Group): {balanced_mean_bmi_control}")
print(f"Balanced Mean Depression (Control Group): {balanced_mean_depression_control}")
print(f"Balanced Mean Employment Status (Control Group): {weighted_mean_employment_status_control}")
print(f"Balanced Mean PA (Control Group): {balanced_mean_PA_control}")
print(f"Balanced Mean QOL (Control Group): {balanced_mean_quality_of_life_control}")
print(f"Balanced Mean Pain (Control Group): {balanced_mean_pain_control}")

print("--------------------------------------------------")
print("Mean in treatnement group:")
print(f"Mean Age (Treatment Group): {mean_age_treatment}")
print(f"Mean Gender (Treatment Group): {mean_gender_control}")
print(f"Mean BMI (Treatment Group): {mean_bmi_treatment}")
print(f"Mean Depression (Treatment Group): {mean_Depression_treatment}")
print(f"Mean Employment Status (Treatment Group): {mean_employment_status_treatment}")
print(f"Mean Physical Activity (Treatment Group): {mean_PA_treatment}")
print(f"Mean QOL (Treatment Group): {mean_QOL_treatment}")
print(f"Mean Pain (Treatment Group): {mean_pain_treatment}")

Mean in control group:
Mean Age (Control Group): 61.1583044982699
Mean Gender (Control Group): 1.7002499013547283
Mean BMI (Control Group): 30.10155709342561
Mean Depression (Control Group): 1.3079584775086506
Mean Employment Status (Control Group): 1.694636678200692
Mean Physical Activity (Control Group): 1.7664359861591696
Mean QOL (Control Group): 51.83282871972318
Mean Pain (Control Group): 50.36332179930796
--------------------------------------------------
Weighted mean in control group:
Weighted Mean Age (Control Group): 56.693414160665185
Weighted Mean Gender (Control Group): 1.6483526145577174
Weighted Mean BMI (Control Group): 29.594480125628465
Weighted Mean Depression (Control Group): 1.1206600852591877
Weighted Mean Employment Status (Control Group): 1.1677992003696231
Weighted Mean PA (Control Group): 1.7664359861591696
Weighted Mean QOL (Control Group): 45.05659058318614
Weighted Mean Pain (Control Group): 47.219554878553055
----------------------------------------------

In [74]:
balanced_control_data.to_csv('testdata1/3.PA_balanced.csv', index=False)

In [75]:
balanced_control_data

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Pain,Baseline_Quality_of_life,PA_change,Treatment
0,68.0,1.0,31.6,1.0,1.0,1,30.0,56.25,0,0
1,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
2,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
3,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
4,46.0,1.0,30.0,1.0,1.0,1,20.0,68.75,0,0
...,...,...,...,...,...,...,...,...,...,...
1151,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1152,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1153,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1154,59.0,1.0,34.1,1.0,1.0,3,90.0,37.50,0,0


In [76]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Pain,Baseline_Quality_of_life,PA_change,Treatment
0,68.0,1.0,31.6,1.0,1.0,1,30.0,56.25,0,0
1,45.0,2.0,32.3,4.0,2.0,1,90.0,0.00,0,0
2,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
3,46.0,1.0,30.0,1.0,1.0,1,20.0,68.75,0,0
4,75.0,1.0,27.8,1.0,3.0,1,90.0,56.25,0,0
...,...,...,...,...,...,...,...,...,...,...
1151,57.0,2.0,31.8,1.0,1.0,3,80.0,50.00,0,0
1152,67.0,1.0,23.1,2.0,1.0,2,70.0,75.00,1,0
1153,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1154,59.0,1.0,34.1,1.0,1.0,3,90.0,37.50,0,0


In [77]:
x = pd.read_csv('testdata1/3.PA_balanced.csv')

In [78]:
x

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Pain,Baseline_Quality_of_life,PA_change,Treatment
0,68.0,1.0,31.6,1.0,1.0,1,30.0,56.25,0,0
1,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
2,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
3,59.0,2.0,26.8,1.0,1.0,2,70.0,37.50,-1,0
4,46.0,1.0,30.0,1.0,1.0,1,20.0,68.75,0,0
...,...,...,...,...,...,...,...,...,...,...
1151,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1152,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1153,61.0,2.0,33.2,1.0,1.0,3,30.0,37.50,0,0
1154,59.0,1.0,34.1,1.0,1.0,3,90.0,37.50,0,0


In [79]:
balanced_control_data.PA_change.value_counts()

PA_change
 0    653
-1    218
 1    146
-2     77
 2     62
Name: count, dtype: int64

In [80]:
df.PA_change.value_counts()

PA_change
 0    708
-1    188
 1    147
-2     57
 2     56
Name: count, dtype: int64